In [52]:
%matplotlib inline
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import csv

# Data Cleanning

## NYPD Complaints

In [94]:
nypd_complaint = pd.read_csv('data/NYPD_Complaint_Data.csv')

In [95]:
nypd_complaint.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,885776788,66,NaN,12/23/2020,19:50:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,18-24,BLACK,M,986633,167258,40.625769,-73.991417,"(40.62576896100006, -73.99141682199996)",POINT (-73.99141682199996 40.62576896100006)
1,350637195,77,NaN,12/21/2020,01:10:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,M,1003606,185050,40.674583,-73.930222,"(40.67458330800008, -73.93022154099998)",POINT (-73.93022154099998 40.67458330800008)
2,347843168,43,BRONX,11/22/2020,22:00:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,F,1020316,239179,40.823101,-73.869690,"(40.82310129900002, -73.86969046099993)",POINT (-73.86969046099993 40.82310129900002)
3,197941396,47,NaN,11/22/2020,09:50:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,F,1026387,262634,40.887451,-73.847608,"(40.88745131300004, -73.84760778699997)",POINT (-73.84760778699997 40.88745131300004)
4,298404927,25,NaN,11/21/2020,15:38:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,18-24,BLACK HISPANIC,M,1003396,230824,40.800222,-73.930848,"(40.80022202900005, -73.93084834199995)",POINT (-73.93084834199995 40.80022202900005)


In [106]:
nypd_complaint = nypd_complaint[['CMPLNT_FR_DT', 'LAW_CAT_CD','SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude']]
nypd_complaint = nypd_complaint.rename({'CMPLNT_FR_DT': 'date', 'LAW_CAT_CD': 'level'}, axis='columns').rename(str.lower, axis='columns')
nypd_complaint

,date,level,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,latitude,longitude
0,12/23/2020,FELONY,NaN,NaN,NaN,18-24,BLACK,M,40.625769,-73.991417
1,12/21/2020,FELONY,NaN,NaN,NaN,25-44,BLACK,M,40.674583,-73.930222
2,11/22/2020,FELONY,UNKNOWN,UNKNOWN,U,25-44,BLACK,F,40.823101,-73.869690
3,11/22/2020,FELONY,25-44,BLACK,M,25-44,BLACK,F,40.887451,-73.847608
4,11/21/2020,FELONY,NaN,NaN,NaN,18-24,BLACK HISPANIC,M,40.800222,-73.930848
...,...,...,...,...,...,...,...,...,...,...
413407,01/04/2020,MISDEMEANOR,UNKNOWN,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,D,40.741341,-73.978393
413408,01/02/2020,FELONY,UNKNOWN,UNKNOWN,U,45-64,ASIAN / PACIFIC ISLANDER,M,40.688716,-73.826366
413409,01/02/2020,FELONY,25-44,BLACK,M,UNKNOWN,UNKNOWN,D,40.789947,-73.975354
413410,01/05/2020,MISDEMEANOR,45-64,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,E,40.882338,-73.891652


### we are going to compare the data between 2019 and 2020, therefore, we need to remove other data

In [116]:
nypd_complaint_2019 = nypd_complaint[nypd_complaint['date'].str.contains('2019')].reset_index(drop=True)
nypd_complaint_2019

,date,level,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,latitude,longitude
0,05/08/2019,FELONY,45-64,BLACK,M,65+,BLACK,M,40.823101,-73.869690
1,11/24/2019,FELONY,25-44,BLACK,M,25-44,BLACK,M,40.822570,-73.911307
2,05/07/2019,MISDEMEANOR,45-64,WHITE HISPANIC,M,45-64,BLACK,F,40.849712,-73.830373
3,06/17/2019,FELONY,NaN,NaN,NaN,45-64,WHITE HISPANIC,M,40.715414,-73.905945
4,07/17/2019,VIOLATION,45-64,BLACK,F,25-44,BLACK,F,40.666796,-73.896512
...,...,...,...,...,...,...,...,...,...,...
6562,12/13/2019,FELONY,25-44,BLACK,M,25-44,WHITE,M,40.795695,-73.938737
6563,12/24/2019,MISDEMEANOR,UNKNOWN,WHITE,M,45-64,WHITE,F,40.634049,-74.076606
6564,12/24/2019,FELONY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,D,40.776140,-73.910701
6565,12/05/2019,FELONY,NaN,NaN,NaN,45-64,WHITE HISPANIC,M,40.862924,-73.893857


In [117]:
nypd_complaint_2020 = nypd_complaint[nypd_complaint['date'].str.contains('2020')].reset_index(drop=True)
nypd_complaint_2020

,date,level,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,latitude,longitude
0,12/23/2020,FELONY,NaN,NaN,NaN,18-24,BLACK,M,40.625769,-73.991417
1,12/21/2020,FELONY,NaN,NaN,NaN,25-44,BLACK,M,40.674583,-73.930222
2,11/22/2020,FELONY,UNKNOWN,UNKNOWN,U,25-44,BLACK,F,40.823101,-73.869690
3,11/22/2020,FELONY,25-44,BLACK,M,25-44,BLACK,F,40.887451,-73.847608
4,11/21/2020,FELONY,NaN,NaN,NaN,18-24,BLACK HISPANIC,M,40.800222,-73.930848
...,...,...,...,...,...,...,...,...,...,...
404887,01/04/2020,MISDEMEANOR,UNKNOWN,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,D,40.741341,-73.978393
404888,01/02/2020,FELONY,UNKNOWN,UNKNOWN,U,45-64,ASIAN / PACIFIC ISLANDER,M,40.688716,-73.826366
404889,01/02/2020,FELONY,25-44,BLACK,M,UNKNOWN,UNKNOWN,D,40.789947,-73.975354
404890,01/05/2020,MISDEMEANOR,45-64,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,E,40.882338,-73.891652


### We can see that there are lots of NaN values, we can fill them with 'UNKNOWN' instead

In [119]:
nypd_complaint_2019 = nypd_complaint_2019.fillna('UNKNOWN')
nypd_complaint_2020 = nypd_complaint_2020.fillna('UNKNOWN')
nypd_complaint_2020

,date,level,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,latitude,longitude
0,12/23/2020,FELONY,UNKNOWN,UNKNOWN,UNKNOWN,18-24,BLACK,M,40.625769,-73.991417
1,12/21/2020,FELONY,UNKNOWN,UNKNOWN,UNKNOWN,25-44,BLACK,M,40.674583,-73.930222
2,11/22/2020,FELONY,UNKNOWN,UNKNOWN,U,25-44,BLACK,F,40.823101,-73.869690
3,11/22/2020,FELONY,25-44,BLACK,M,25-44,BLACK,F,40.887451,-73.847608
4,11/21/2020,FELONY,UNKNOWN,UNKNOWN,UNKNOWN,18-24,BLACK HISPANIC,M,40.800222,-73.930848
...,...,...,...,...,...,...,...,...,...,...
404887,01/04/2020,MISDEMEANOR,UNKNOWN,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,D,40.741341,-73.978393
404888,01/02/2020,FELONY,UNKNOWN,UNKNOWN,U,45-64,ASIAN / PACIFIC ISLANDER,M,40.688716,-73.826366
404889,01/02/2020,FELONY,25-44,BLACK,M,UNKNOWN,UNKNOWN,D,40.789947,-73.975354
404890,01/05/2020,MISDEMEANOR,45-64,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,E,40.882338,-73.891652


In [121]:
nypd_complaint_2020['level'].value_counts().rename_axis('level').reset_index(name='counts')

,type,counts
0,MISDEMEANOR,207627
1,FELONY,130794
2,VIOLATION,66471


### check age groups

In [125]:
nypd_complaint_2020['susp_age_group'].value_counts().rename_axis('susp_age_group').reset_index(name='counts')

,susp_age_group,counts
0,UNKNOWN,234081
1,25-44,98792
2,45-64,33155
3,18-24,29185
4,<18,6463
5,65+,3196
6,2020,10
7,2019,2
8,-962,1
9,-12,1


### we can see that there are some unreasonable age groups, since it is difficult to understand its original meanings, we decide to remove all these rows.

In [127]:
nypd_complaint_2020 = nypd_complaint_2020[~nypd_complaint_2020['susp_age_group'].isin(['2020', '2019', '-962', '-12', '-71', '1925', '-965', '-942', '-977', '1020'])]
nypd_complaint_2020['susp_age_group'].value_counts().rename_axis('susp_age_group').reset_index(name='counts')


,susp_age_group,counts
0,UNKNOWN,234081
1,25-44,98792
2,45-64,33155
3,18-24,29185
4,<18,6463
5,65+,3196


In [131]:
nypd_complaint_2020['susp_race'].value_counts().rename_axis('susp_race').reset_index(name='counts')

,susp_race,counts
0,UNKNOWN,185806
1,BLACK,113846
2,WHITE HISPANIC,49784
3,WHITE,28476
4,BLACK HISPANIC,15720
5,ASIAN / PACIFIC ISLANDER,10633
6,AMERICAN INDIAN/ALASKAN NATIVE,607


In [132]:
nypd_complaint_2020['susp_sex'].value_counts().rename_axis('susp_sex').reset_index(name='counts')

,susp_sex,counts
0,M,183152
1,UNKNOWN,92098
2,U,81033
3,F,48589


### We can see that 'U' should represent 'UNKNOWN', so we can merge these two types to 'Others'.

In [140]:
nypd_complaint_2020 = nypd_complaint_2020.replace({'susp_sex': {'U': 'Others', 'UNKNOWN': 'Others'}})
nypd_complaint_2020['susp_sex'].value_counts().rename_axis('susp_sex').reset_index(name='counts')

,susp_sex,counts
0,M,183140
1,Others,173125
2,F,48586


### We are going to do the same thing for vic's data

In [135]:
nypd_complaint_2020['vic_age_group'].value_counts().rename_axis('vic_age_group').reset_index(name='counts')

,susp_sex,counts
0,25-44,156856
1,UNKNOWN,98647
2,45-64,82828
3,18-24,36948
4,65+,18288
5,<18,11284
6,-948,2
7,-938,1
8,-31,1
9,950,1


In [136]:
nypd_complaint_2020 = nypd_complaint_2020[nypd_complaint_2020['vic_age_group'].isin(['25-44', 'UNKNOWN', '45-64', '18-24', '65+', '<18'])]
nypd_complaint_2020['vic_age_group'].value_counts().rename_axis('vic_age_group').reset_index(name='counts')


,vic_age_group,counts
0,25-44,156856
1,UNKNOWN,98647
2,45-64,82828
3,18-24,36948
4,65+,18288
5,<18,11284


In [137]:
nypd_complaint_2020['vic_race'].value_counts().rename_axis('vic_race').reset_index(name='counts')

,vic_race,counts
0,UNKNOWN,107825
1,BLACK,107637
2,WHITE HISPANIC,74680
3,WHITE,64170
4,ASIAN / PACIFIC ISLANDER,31621
5,BLACK HISPANIC,17619
6,AMERICAN INDIAN/ALASKAN NATIVE,1299


In [138]:
nypd_complaint_2020['vic_sex'].value_counts().rename_axis('vic_sex').reset_index(name='counts')

,vic_sex,counts
0,F,160060
1,M,149569
2,D,62317
3,E,32904
4,UNKNOWN,1


### Since we have no idea what is sex D or E, we decided to mark them as 'Others'.

In [142]:
nypd_complaint_2020 = nypd_complaint_2020.replace({'vic_sex': {'D': 'Others', 'E': 'Others', 'UNKNOWN': 'Others'}})
nypd_complaint_2020['vic_sex'].value_counts().rename_axis('vic_sex').reset_index(name='counts')

,vic_sex,counts
0,F,160060
1,M,149569
2,Others,95222


In [143]:
nypd_complaint_2020

,date,level,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,latitude,longitude
0,12/23/2020,FELONY,UNKNOWN,UNKNOWN,Others,18-24,BLACK,M,40.625769,-73.991417
1,12/21/2020,FELONY,UNKNOWN,UNKNOWN,Others,25-44,BLACK,M,40.674583,-73.930222
2,11/22/2020,FELONY,UNKNOWN,UNKNOWN,Others,25-44,BLACK,F,40.823101,-73.869690
3,11/22/2020,FELONY,25-44,BLACK,M,25-44,BLACK,F,40.887451,-73.847608
4,11/21/2020,FELONY,UNKNOWN,UNKNOWN,Others,18-24,BLACK HISPANIC,M,40.800222,-73.930848
...,...,...,...,...,...,...,...,...,...,...
404887,01/04/2020,MISDEMEANOR,UNKNOWN,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,Others,40.741341,-73.978393
404888,01/02/2020,FELONY,UNKNOWN,UNKNOWN,Others,45-64,ASIAN / PACIFIC ISLANDER,M,40.688716,-73.826366
404889,01/02/2020,FELONY,25-44,BLACK,M,UNKNOWN,UNKNOWN,Others,40.789947,-73.975354
404890,01/05/2020,MISDEMEANOR,45-64,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,Others,40.882338,-73.891652


### And we are going to do the same steps for 2019 complaints

In [144]:
nypd_complaint_2019['level'].value_counts().rename_axis('level').reset_index(name='counts')

,level,counts
0,FELONY,3022
1,MISDEMEANOR,2888
2,VIOLATION,657


In [145]:
nypd_complaint_2019['susp_age_group'].value_counts().rename_axis('susp_age_group').reset_index(name='counts')

,level,counts
0,UNKNOWN,4350
1,25-44,1153
2,45-64,518
3,18-24,365
4,<18,130
5,65+,50
6,-928,1


In [147]:
nypd_complaint_2019 = nypd_complaint_2019[nypd_complaint_2019['susp_age_group'] != '-928']
nypd_complaint_2019['susp_age_group'].value_counts().rename_axis('vic_age_group').reset_index(name='counts')


,vic_age_group,counts
0,UNKNOWN,4350
1,25-44,1153
2,45-64,518
3,18-24,365
4,<18,130
5,65+,50


In [148]:
nypd_complaint_2019['susp_race'].value_counts().rename_axis('susp_race').reset_index(name='counts')

,susp_race,counts
0,UNKNOWN,4090
1,BLACK,1071
2,WHITE HISPANIC,539
3,WHITE,518
4,BLACK HISPANIC,183
5,ASIAN / PACIFIC ISLANDER,154
6,AMERICAN INDIAN/ALASKAN NATIVE,11


In [149]:
nypd_complaint_2019['susp_sex'].value_counts().rename_axis('susp_sex').reset_index(name='counts')

,susp_sex,counts
0,UNKNOWN,2297
1,M,2115
2,U,1489
3,F,665


In [150]:
nypd_complaint_2019 = nypd_complaint_2019.replace({'susp_sex': {'U': 'Others','UNKNOWN': 'Others'}})
nypd_complaint_2019['susp_sex'].value_counts().rename_axis('vic_sex').reset_index(name='counts')

,vic_sex,counts
0,Others,3786
1,M,2115
2,F,665


In [151]:
nypd_complaint_2019['vic_age_group'].value_counts().rename_axis('vic_age_group').reset_index(name='counts')

,vic_age_group,counts
0,25-44,2306
1,45-64,1403
2,UNKNOWN,1349
3,65+,553
4,18-24,509
5,<18,446


In [152]:
nypd_complaint_2019['vic_race'].value_counts().rename_axis('vic_race').reset_index(name='counts')

,vic_race,counts
0,UNKNOWN,1651
1,BLACK,1543
2,WHITE,1522
3,WHITE HISPANIC,1054
4,ASIAN / PACIFIC ISLANDER,525
5,BLACK HISPANIC,249
6,AMERICAN INDIAN/ALASKAN NATIVE,22


In [153]:
nypd_complaint_2019['vic_sex'].value_counts().rename_axis('vic_sex').reset_index(name='counts')

,vic_sex,counts
0,F,3149
1,M,2104
2,D,750
3,E,563


In [154]:
nypd_complaint_2019 = nypd_complaint_2019.replace({'susp_sex': {'D': 'Others','E': 'Others'}})
nypd_complaint_2019['susp_sex'].value_counts().rename_axis('vic_sex').reset_index(name='counts')

,vic_sex,counts
0,Others,3786
1,M,2115
2,F,665


In [155]:
nypd_complaint_2019

,date,level,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,latitude,longitude
0,05/08/2019,FELONY,45-64,BLACK,M,65+,BLACK,M,40.823101,-73.869690
1,11/24/2019,FELONY,25-44,BLACK,M,25-44,BLACK,M,40.822570,-73.911307
2,05/07/2019,MISDEMEANOR,45-64,WHITE HISPANIC,M,45-64,BLACK,F,40.849712,-73.830373
3,06/17/2019,FELONY,UNKNOWN,UNKNOWN,Others,45-64,WHITE HISPANIC,M,40.715414,-73.905945
4,07/17/2019,VIOLATION,45-64,BLACK,F,25-44,BLACK,F,40.666796,-73.896512
...,...,...,...,...,...,...,...,...,...,...
6562,12/13/2019,FELONY,25-44,BLACK,M,25-44,WHITE,M,40.795695,-73.938737
6563,12/24/2019,MISDEMEANOR,UNKNOWN,WHITE,M,45-64,WHITE,F,40.634049,-74.076606
6564,12/24/2019,FELONY,UNKNOWN,UNKNOWN,Others,UNKNOWN,UNKNOWN,D,40.776140,-73.910701
6565,12/05/2019,FELONY,UNKNOWN,UNKNOWN,Others,45-64,WHITE HISPANIC,M,40.862924,-73.893857
